In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from numpy import isnan
from pandas import read_csv

In [2]:
# Read all six Excel files into separate DataFrames
ORDER_RECIPE1 = pd.read_excel('ORDER_RECIPE1.xls')
ORDER_RECIPE2 = pd.read_excel('ORDER_RECIPE2.xls')
ORDER_RECIPE3 = pd.read_excel('ORDER_RECIPE3.xls')
ORDER_RECIPE4 = pd.read_excel('ORDER_RECIPE4.xls')
ORDER_RECIPE5 = pd.read_excel('ORDER_RECIPE5.xls')
ORDER_RECIPE6 = pd.read_excel('ORDER_RECIPE6.xls')
ORDER_RECIPE7 = pd.read_excel('ORDER_RECIPE7.xls')
ORDER_RECIPE8 = pd.read_excel('ORDER_RECIPE8.xls')
ORDER_RECIPE = pd.concat([ORDER_RECIPE1, ORDER_RECIPE2, ORDER_RECIPE3, ORDER_RECIPE4, ORDER_RECIPE5, ORDER_RECIPE6,ORDER_RECIPE7,ORDER_RECIPE8], ignore_index=True)

In [3]:
columns_to_keep_oss = ['ORDER_ID','ORDER_RECIPE_ID','PRODUCT_GROUP_CODE','ORDERED_QUANTITY','TERMINAL_PRODUCT_NUMBER']
ORDER_RECIPE = ORDER_RECIPE[columns_to_keep_oss]
ORDER_RECIPE = ORDER_RECIPE[ORDER_RECIPE['ORDERED_QUANTITY'] != 0]
ORDER_RECIPE = ORDER_RECIPE.dropna()

In [4]:
ORDERS1 = pd.read_excel('ORDERS.xls')
ORDERS2 = pd.read_excel('ORDERS2.xls')

ORDERS = pd.concat([ORDERS1, ORDERS2], ignore_index=True)

In [5]:
ORDERS.columns

Index(['ORDER_ID', 'LOADED_DRIVER_NUMBER', 'USER_NAME',
       'TRAILER_CARRIER_NUMBER', 'CARRIER_NUMBER', 'TRAILER_CODE',
       'VEHICLE_CARRIER_NUMBER', 'VEHICLE_CODE', 'TRACTOR_CARRIER_NUMBER',
       'LOADED_CARRIER_NUMBER',
       ...
       'SIGNATURE_PATH', 'SIGNATURE_FILE_NAME', 'PDXR_AUTHORIZING_SYSTEM',
       'START_TIMESLOT_DATETIME', 'END_TIMESLOT_DATETIME',
       'START_PICKUP_DATETIME', 'END_PICKUP_DATETIME',
       'ADMINISTRATIVE_BOOKING', 'Unnamed: 138', 'LAST_MODIFIED'],
      dtype='object', length=140)

In [6]:
columns_to_keep_osss = ['ORDER_ID','FOLIO_NUMBER','DESTINATION_NUMBER']
ORDERS = ORDERS[columns_to_keep_osss]
ORDERS = ORDERS.dropna()

In [7]:
ORDERS['FOLIO_NUMBER'] = pd.to_datetime(ORDERS['FOLIO_NUMBER'], format='%Y%m%d')

In [8]:
ORDERS

,ORDER_ID,FOLIO_NUMBER,DESTINATION_NUMBER
0,8320,2021-09-27,12
1,12128,2021-09-28,13
2,19110,2021-09-28,12
3,19143,2021-09-28,12
4,19176,2021-09-27,6
...,...,...,...
70198,3098702,2023-07-19,1
70199,3098749,2023-07-19,3
70200,3098806,2023-07-19,3
70201,3098862,2023-07-19,1


In [9]:
merged_data=pd.merge(ORDERS, ORDER_RECIPE, on='ORDER_ID', how='inner')
merged_data = merged_data.sort_values(by='FOLIO_NUMBER')
merged_data = merged_data.reset_index(drop=True)
merged_data

,ORDER_ID,FOLIO_NUMBER,DESTINATION_NUMBER,ORDER_RECIPE_ID,PRODUCT_GROUP_CODE,ORDERED_QUANTITY,TERMINAL_PRODUCT_NUMBER
0,8320,2021-09-27,12,8343,DSL,100,7
1,20678,2021-09-27,13,20705,DSL,2000,7
2,20678,2021-09-27,13,20703,DSL,5000,7
3,20678,2021-09-27,13,20701,DSL,6000,7
4,20678,2021-09-27,13,20699,DSL,10000,7
...,...,...,...,...,...,...,...
493236,3095622,2023-07-19,20,3095645,DSL,3000,7
493237,3095622,2023-07-19,20,3095647,DSL,2000,7
493238,3095622,2023-07-19,20,3095649,DSL,4000,7
493239,3095770,2023-07-19,8,3095807,ADD,400,10


In [10]:
merged_data['ORDERED_QUANTITY'].max()

34080

In [18]:
df = merged_data
merged_data

,ORDER_ID,FOLIO_NUMBER,DESTINATION_NUMBER,ORDER_RECIPE_ID,PRODUCT_GROUP_CODE,ORDERED_QUANTITY,TERMINAL_PRODUCT_NUMBER
0,8320,2021-09-27,12,8343,DSL,100,7
1,20678,2021-09-27,13,20705,DSL,2000,7
2,20678,2021-09-27,13,20703,DSL,5000,7
3,20678,2021-09-27,13,20701,DSL,6000,7
4,20678,2021-09-27,13,20699,DSL,10000,7
...,...,...,...,...,...,...,...
493236,3095622,2023-07-19,20,3095645,DSL,3000,7
493237,3095622,2023-07-19,20,3095647,DSL,2000,7
493238,3095622,2023-07-19,20,3095649,DSL,4000,7
493239,3095770,2023-07-19,8,3095807,ADD,400,10


In [25]:
# Group by PRODUCT_GROUP_CODE and iterate over each group
delivery_list = []
detailed_delivery_list = []

for terminal, group in df.groupby('PRODUCT_GROUP_CODE'):
    # Initialize variables for each group
    current_delivery = []
    current_quantity = 0
    current_order_count = 0
    first_folio = None
    last_folio = None
    list_stations = []  # List to keep track of unique destination numbers
    
    # Iterate over rows in the group
    for index, row in group.iterrows():
        # Check if adding this order exceeds the maximum capacity or the 2-day waiting period
        if (current_quantity + row['ORDERED_QUANTITY'] > 33000) or \
           (last_folio and (row['FOLIO_NUMBER'] - last_folio).days > 2):
            # Append delivery details to the list
            delivery_list.append({
                'PRODUCT_GROUP_CODE': terminal,
                'FIRST_FOLIO_NUMBER': first_folio,
                'LAST_FOLIO_NUMBER': last_folio,
                'ORDER_COUNT': len(list_stations),  # Use count of unique stations
                'TOTAL_QUANTITY': current_quantity
            })
            
            # Update variables for the new delivery
            current_delivery = []
            current_quantity = 0
            current_order_count = 0
            first_folio = None
            last_folio = None
            list_stations = []  # Reset the list for the new delivery
        
        # Add order details to the current delivery
        current_delivery.append(row['FOLIO_NUMBER'])
        current_quantity += row['ORDERED_QUANTITY']
        
        # Update first and last folio numbers
        if first_folio is None:
            first_folio = row['FOLIO_NUMBER']
        last_folio = row['FOLIO_NUMBER']
        
        # Check if the destination number is not in the list_stations, then add it
        if row['DESTINATION_NUMBER'] not in list_stations:
            current_order_count += 1
            list_stations.append(row['DESTINATION_NUMBER'])
        
        # Append detailed delivery information to the list
        detailed_delivery_list.append({
            'PRODUCT_GROUP_CODE': terminal,
            'DELIVERY_ID': len(delivery_list),  # Use delivery index as delivery ID
            'FOLIO_NUMBER': row['FOLIO_NUMBER'],
            'DESTINATION_NUMBER': row['DESTINATION_NUMBER'],
            'QUANTITY_ORDERED': row['ORDERED_QUANTITY']
        })

    # Append details of the last delivery for this group
    delivery_list.append({
        'PRODUCT_GROUP_CODE': terminal,
        'FIRST_FOLIO_NUMBER': first_folio,
        'LAST_FOLIO_NUMBER': last_folio,
        'ORDER_COUNT': len(list_stations),  # Use count of unique stations
        'TOTAL_QUANTITY': current_quantity
    })

# Create DataFrame from delivery list
delivery_df = pd.DataFrame(delivery_list)

# Create DataFrame from detailed delivery list
delivery_df_detailed = pd.DataFrame(detailed_delivery_list)


In [26]:
delivery_df

,PRODUCT_GROUP_CODE,FIRST_FOLIO_NUMBER,LAST_FOLIO_NUMBER,ORDER_COUNT,TOTAL_QUANTITY
0,ADD,2021-09-27,2021-09-27,2,33000
1,ADD,2021-09-27,2021-09-27,2,32950
2,ADD,2021-09-27,2021-09-27,2,32575
3,ADD,2021-09-27,2021-09-28,2,32825
4,ADD,2021-09-28,2021-09-28,2,32625
...,...,...,...,...,...
73874,GAS,2023-07-19,2023-07-19,2,33000
73875,GAS,2023-07-19,2023-07-19,2,31000
73876,GAS,2023-07-19,2023-07-19,5,32000
73877,GAS,2023-07-19,2023-07-19,4,33000


In [27]:
delivery_df_detailed

,PRODUCT_GROUP_CODE,DELIVERY_ID,FOLIO_NUMBER,DESTINATION_NUMBER,QUANTITY_ORDERED
0,ADD,0,2021-09-27,1,2700
1,ADD,0,2021-09-27,1,900
2,ADD,0,2021-09-27,1,600
3,ADD,0,2021-09-27,8,700
4,ADD,0,2021-09-27,8,1500
...,...,...,...,...,...
493236,GAS,73878,2023-07-19,20,7000
493237,GAS,73878,2023-07-19,20,6000
493238,GAS,73878,2023-07-19,1,3000
493239,GAS,73878,2023-07-19,12,4000


In [28]:
grouped_delivery_detailed = delivery_df_detailed.groupby(['PRODUCT_GROUP_CODE', 'DELIVERY_ID', 'FOLIO_NUMBER', 'DESTINATION_NUMBER']).agg({'QUANTITY_ORDERED': 'sum'}).reset_index()
grouped_delivery_detailed

,PRODUCT_GROUP_CODE,DELIVERY_ID,FOLIO_NUMBER,DESTINATION_NUMBER,QUANTITY_ORDERED
0,ADD,0,2021-09-27,1,24300
1,ADD,0,2021-09-27,8,8700
2,ADD,1,2021-09-27,1,9900
3,ADD,1,2021-09-27,8,23050
4,ADD,2,2021-09-27,1,21300
...,...,...,...,...,...
142111,GAS,73877,2023-07-19,12,4000
142112,GAS,73877,2023-07-19,20,3000
142113,GAS,73878,2023-07-19,1,3000
142114,GAS,73878,2023-07-19,12,7000


In [29]:
delivery_df.to_csv('delivery.csv')

In [30]:
grouped_delivery_detailed.to_csv('delivery_detailed.csv')